In [1]:
import pandas as pd

In [7]:
master_df = pd.read_excel(f"./data/master.xlsx")

27820


In [ ]:
print(master_df.columns)
print(master_df.head)

In [ ]:
for column in master_df.columns:
    print(f"{column = }")
    nan_series = master_df[column].isna()
    if True in nan_series[:].tolist():
        print(master_df[column])
        print(f"Total no.of Nan values: {master_df[column].isna().sum()}")

In [2]:
##DataFrame for unit testing
country_name_ls = ['Albania'] *4 + ['Russia']*2 + ['Albania']*2 + ['India']*2
year_ls = ['2009', '2009','2010', '2011', '2011', '2012', '2010', '2012', '2009', '2010']
sex_ls = ['m']*4 + ['f']*2 + ['m'] + ['f']*3
test_df = pd.DataFrame()
test_df['Country'] = country_name_ls
test_df['Year'] = year_ls
test_df['Sex'] = sex_ls

In [3]:
class preprocess_df:
  '''
  Class to preprocess DataFrame
  '''
  def encode_attributes(self, input_df, column_indices):
    transformed_dicts_ls = []
    transformed_df = input_df.copy(deep = True)
    column_names = transformed_df.columns.tolist()
    print(f"{column_names = }")
    for col_iter in column_indices:
      temp_dict = {}
      temp_key = 0
      temp_ls = []
      column_name = column_names[col_iter]
      for col in transformed_df.iloc[:,col_iter].tolist():
        if col not in [*temp_dict.keys()]:
          temp_dict[col] = temp_key
          temp_key += 1
        temp_ls.append(temp_dict[col])
      dict_inv = {v:k for k,v in temp_dict.items()}
      transformed_dicts_ls.append(dict_inv)
      transformed_df[column_name] = temp_ls
    return transformed_df, transformed_dicts_ls

In [10]:
#BUC implementation
class buc:
    def __init__(self, df, column_enc_dicts_ls):
        self.numDims = df.shape[1]
        self.cardinality = []
        self.minsup = 1
        self.output_df = None
        self.datacounts = [[], [], []]
        self.attribute_ls = ["*"] * df.shape[1]
        self.debug_counter = 0
        self.output_dict = {}
        self.column_enc_dicts_ls = column_enc_dicts_ls

    def counting_sort(self, array_a, df_idx_ls):
      array_c = [0]*(max(array_a) + 1)
      idx_ls = [-1] * (len(array_a))

      # print(f"{array_a = }")
      # print(f"{array_c = }")
      for i in range(0, len(array_a)):
        array_c[array_a[i]] += 1

      for i in range(0, len(array_c) - 1):
        array_c[i+1] = array_c[i] + array_c[i+1]

      for i in range(len(array_a) - 1, -1, -1):
        array_c[array_a[i]] = array_c[array_a[i]] - 1
        idx = array_c[array_a[i]]
        idx_ls[idx] = df_idx_ls[i]

      # idx_ls = [i + min_idx for i in idx_ls]
      # print(f"{array_a = }")
      # print(f"{idx_ls = }")
      return idx_ls


    def partition(self, input_df, d, bigc):
        #Sorting the dataframe
        temp_counter_dict = {}
        sorted_idx = self.counting_sort(input_df.iloc[:,d].tolist(), input_df.index.tolist())
        input_df = input_df.reindex(sorted_idx)
        for attribute in input_df.iloc[:,d].tolist():
            temp_counter_dict[attribute] = temp_counter_dict.get(attribute, 0) + 1
        self.datacounts[d] = [*temp_counter_dict.values()]
        # self.datacounts[d] = input_df.iloc[:,d].value_counts().tolist()
        # print(f"{self.datacounts = }")
        return input_df

    def buc_implementation(self, input, dim):
        self.debug_counter += 1
        # print(f"iter: {self.debug_counter - 1}")
        # print(f"Aggregate: {input.shape[0]}")
        if tuple(self.attribute_ls) in [*self.output_dict.keys()]:
          print(f"Error!!")
        self.output_dict[tuple(self.attribute_ls)] = input.shape[0]
        # print(f"{self.output_dict =}")
        # if self.debug_counter == 12:
          # return 
        # print(f"{dim = }")
        for d in range(dim, self.numDims,1):
            bigc = input.iloc[:,d].nunique()
            # print(f"{d= }, {bigc=}")
            # print(f"Input before partitioning: {input}")
            input = self.partition(input, d, bigc)
            # print(f"Input after partitioning on {d}: {input}")
            # print(f"{self.datacounts = }")
            k = 0
            for i in range(0, bigc, 1):
                # print(f"################Inside i loop######################")
                # print(f"{d = }, {i = }")
                # print(f"{self.datacounts = }")
                smallc = self.datacounts[d][i]
                # print(f"{smallc = }")
                if smallc >= self.minsup:
                    # print(f"**********************Inside if condition***********************")
                    # print(f"k, d: {k,d}")
                    # print(f"Attribute: {input.iloc[k,d]}")
                    # print(f"{transformed_dicts[d] = }")
                    self.attribute_ls[d] = self.column_enc_dicts_ls[d][input.iloc[k,d]]
                    self.buc_implementation(input.iloc[k:k+smallc,:], dim=d+1)
                    # if self.debug_counter == 12:
                      # return
                    # print(f"d inside if condition: {d}")
                    # print(f"******************************************************************")
                k += smallc
            # print(f"#################################################################")
            # print(f"ALL:")
            self.attribute_ls[d] = "*"

In [14]:
input_df = test_df
preprocess_obj = preprocess_df()
transformed_df, column_enc_dicts_ls = preprocess_obj.encode_attributes(test_df, [*range(0,test_df.shape[1])]) #NOTE: This should be modified as required
print(f"test_df: {test_df}")
print(f"transformed_df: {transformed_df}")
buc_obj = buc(transformed_df, column_enc_dicts_ls)
buc_obj.buc_implementation(transformed_df, 0)
output_dict = buc_obj.output_dict
# output_df = pd.DataFrame(columns=input_df.columns.tolist())
# for key, value in output_dict:
    
# print(f"output_df: {output_df}")
# print(f"output_dict: {output_dict}")


column_names = ['Country', 'Year', 'Sex']
test_df:    Country  Year Sex
0  Albania  2009   m
1  Albania  2009   m
2  Albania  2010   m
3  Albania  2011   m
4   Russia  2011   f
5   Russia  2012   f
6  Albania  2010   m
7  Albania  2012   f
8    India  2009   f
9    India  2010   f
transformed_df:    Country  Year  Sex
0        0     0    0
1        0     0    0
2        0     1    0
3        0     2    0
4        1     2    1
5        1     3    1
6        0     1    0
7        0     3    1
8        2     0    1
9        2     1    1
output_df: Empty DataFrame
Columns: [Country, Year, Sex]
Index: []
output_dict: {('*', '*', '*'): 10, ('Albania', '*', '*'): 6, ('Albania', '2009', '*'): 2, ('Albania', '2009', 'm'): 2, ('Albania', '2010', '*'): 2, ('Albania', '2010', 'm'): 2, ('Albania', '2011', '*'): 1, ('Albania', '2011', 'm'): 1, ('Albania', '2012', '*'): 1, ('Albania', '2012', 'f'): 1, ('Albania', '*', 'm'): 5, ('Albania', '*', 'f'): 1, ('Russia', '*', '*'): 2, ('Russia', '2011', '*')